In [1]:
from fastai import datasets
import numpy as np
import dynet_config
dynet_config.set(autobatch=True)
import dynet as dy

In [3]:
path = datasets.download_data()
path

TypeError: download_data() missing 1 required positional argument: 'url'

In [27]:
!curl https://s3.amazonaws.com/fast-ai-nlp/wikitext-2.tgz -o wikitext.tgz

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 3974k  100 3974k    0     0  4654k      0 --:--:-- --:--:-- --:--:-- 4654k


In [33]:
!rm wikitext
!mkdir wikitext
!tar -xzvf wikitext.tgz -C wikitext

wikitext-2/
wikitext-2/train.csv
wikitext-2/test.csv


In [35]:
!more wikitext/wikitext-2/train.csv

" 
 = 2013 – 14 York City F.C. season = 
 
 The 2013 – 14 season was the <unk> season of competitive association football a
nd 77th season in the Football League played by York City Football Club , a prof
essional football club based in York , North Yorkshire , England . Their 17th @-
@ place finish in 2012 – 13 meant it was their second consecutive season in Leag
ue Two . The season ran from 1 July 2013 to 30 June 2014 . 
 Nigel Worthington , starting his first full season as York manager , made eight
 permanent summer signings . By the turn of the year York were only above the re
legation zone on goal difference , before a 17 @-@ match unbeaten run saw the te
am finish in seventh @-@ place in the 24 @-@ team 2013 – 14 Football League Two 
. This meant York qualified for the play @-@ offs , and they were eliminated in 
the semi @-@ final by Fleetwood Town . York were knocked out of the 2013 – 14 FA
 Cup , Football League Cup and Football League Trophy in their opening round mat
ches .

In [2]:
class Dataset:
    def __init__(self, xs, ys, shuffle=True):
        assert len(xs) == len(ys)
        self.xs, self.ys = xs, ys
        
    def __len__(self):
        return len(self.xs)
    def __getitem__(self, i):
        try:
            return self.xs[i], self.ys[i]
        except TypeError:
            return list(zip(*[self[_i] for _i in i]))
    def __iter__(self):
        if self.shuffle: idxs = np.random.permutation(len(self))
        else:       idxs = list(range(len(self)))
        for i in range(len(self)):
            yield self[idxs[i]]
    def iter_batches(self, bs=64, shuffle=True):
        if shuffle: idxs = np.random.permutation(len(self))
        else:       idxs = list(range(len(self)))
        batch_start = 0
        while batch_start<len(self):
            yield self[batch_start:batch_start+bs]
            batch_start += bs
            
    def dataloader(self, bs=64, shuffle=True):
        return Dataloader(self, bs, shuffle)
            
    

class Dataloader:
    def __init__(self, dataset, bs=64, shuffle=True):
        self.ds = dataset
        self.bs = bs
        self.shuffle = shuffle
        
    def __len__(self):
        return int(np.ceil(len(self.ds)/self.bs))
        
    def __iter__(self):
        if self.shuffle: idxs = np.random.permutation(len(self.ds))
        else:       idxs = list(range(len(self.ds)))
        batch_start = 0
        while batch_start<len(self.ds):
            bx, by = self.ds[idxs[batch_start:batch_start+self.bs]]
            yield list(bx), by
            batch_start += self.bs
        
class Databunch:
    def __init__(self, train_ds, valid_ds, test_ds=None, bs=64):
        self.train_ds, self.valid_ds, self.test_ds = train_ds, valid_ds, test_ds
        self.train_dl = self.train_ds.dataloader(bs, shuffle=True)
        self.valid_dl = self.valid_ds.dataloader(bs, shuffle=False)
        if self.test_ds:
            self.test_dl = self.test_ds.iter_batches(bs, shuffle=False)

In [3]:
sizes = np.random.randint(50, 80, 100)
data = [np.random.randint(0, 1000, sz) for sz in sizes]
data 

[array([657, 549, 695, 413, ..., 222, 574, 610, 414]),
 array([876, 906, 713,   0, ..., 372, 406, 380, 409]),
 array([182, 252, 515, 449, ..., 975, 547,  93, 144]),
 array([485, 518, 978, 637, ..., 132, 786, 205, 361]),
 array([331,  49, 101, 359, ...,  67, 246, 762, 278]),
 array([150,  92, 903, 507, ..., 213, 540, 970, 940]),
 array([822, 468, 422,  49, ..., 122,  71, 546,  88]),
 array([892, 805,  62, 500, ...,  94, 420, 674, 554]),
 array([726, 528, 838,  97, ..., 562, 509, 196, 311]),
 array([436, 703, 181, 829, ..., 331, 832, 600, 475]),
 array([873, 556, 400,  78, ..., 960, 687, 723, 857]),
 array([122, 134, 490, 356, ..., 440, 836, 889,  48]),
 array([538, 412, 290, 426, ..., 367, 612, 995, 938]),
 array([611, 854, 930, 458, ..., 681, 974, 244, 398]),
 array([668, 271, 614, 703, ..., 937, 205, 287, 850]),
 array([266, 260, 198, 804, 121, 905, 783,  89, 825,  10,  41, 549, 922, 730, 520, 699, 505, 506, 504, 544, 119, 373,
        586, 765, 506, 912, 724, 878, 658, 644,  41, 819,

In [4]:
ds = Dataset(data, data)
dl = ds.dataloader()
bx, by = next(iter(dl))

In [5]:
from fastai_dynet.models import *

In [6]:
class RNN(Module):
    def __init__(self, n_layers, n_in, n_hid, parent=None, name=None, **kwargs):
        super().__init__(parent, name, **kwargs)
        self.n_layers, self.n_in, self.n_hid = n_layers, n_in, n_hid
        self.builder = dy.SimpleRNNBuilder(n_layers, n_in, n_hid, self.params)
        
    def __call__(self, xs):
        state = self.builder.initial_state([dy.zeros(self.n_hid) for _ in range(self.n_layers)])
        self.state = state
        return state.transduce(xs)
    
class RNNLM(Module):
    def __init__(self, n_layers, vocab_sz, emb_dim, n_hid, parent=None, name=None):
        super().__init__(parent, name)
        self.embs = self.params.add_lookup_parameters((vocab_sz, emb_dim))
        self.rnn = RNN(n_layers, emb_dim, n_hid)
        self.out = Linear(n_hid, vocab_sz, activ=dy.softmax)
        
    def __call__(self, xs):
        inp = [dy.lookup(self.embs, x) for x in xs]
        out = self.out(self.rnn(inp))
        return out
        

In [12]:
dy.renew_cg()
m = dy.ParameterCollection()
embs = m.add_lookup_parameters((1000, 25))
out_layer = Linear(32, 1000, None, parent=rnn)

In [13]:
inp = [dy.lookup(embs, i) for i in bx[5]]


In [15]:
rnn = RNN(1, 25, 32, is_top=True)
out = rnn(inp)
out_layer(out)[0].npvalue()

array([ 0.003626, -0.002804,  0.0373  ,  0.043866, ...,  0.040586,  0.027097,  0.004668, -0.046676])

In [69]:
rnn.state.transduce(inp)

ValueError: Bad dimensions for AffineTransform: [{32} {32,25} {1X78} {32,32} {32}]

In [68]:
inp.dim()

((25,), 78)